In [1]:
from dwave.system.samplers import DWaveSampler
from minorminer import find_embedding
from dwave.system.composites import FixedEmbeddingComposite
import numpy as np

In [26]:
solver = "DW_2000Q_6"
sampler = DWaveSampler(solver=solver)
offset0 = 0
offset1 = -0.1
tries = 100

In [27]:
qpu_graph = sampler.edgelist
anneal_offset = np.zeros(2048)
for _ in range(tries):
    embedding = find_embedding(qubo_dict, qpu_graph)
    embedding_idx = [idx for embed_list in embedding.values() for idx in embed_list]
    embed = FixedEmbeddingComposite(sampler, embedding)
    anneal_offset_ranges = np.array(
        embed.properties["child_properties"]["anneal_offset_ranges"]
    )
    offset_range = anneal_offset_ranges[embedding_idx]
    range0 = offset_range[0][0] <= offset0 <= offset_range[0][1]
    range1 = offset_range[1][0] <= offset1 <= offset_range[1][1]
    if range0 and range1:
        anneal_offset[embedding_idx[0]] = offset0
        anneal_offset[embedding_idx[1]] = offset1
        break
print(anneal_offset_ranges[embedding_idx])
print(embedding)
print(qubo_dict)

[[-0.1381342   0.08869053]
 [-0.13777958  0.08278122]]
{0: [502], 1: [510]}
{(0, 0): -0.25, (1, 1): -0.25, (0, 1): 1}


In [28]:
dwave_config = {
    "annealing_time": 1,  # integer microseconds [1, 2000]
    "answer_mode": "histogram",  # histogram or raw
    "auto_scale": False,
    "num_reads": 1000,  # raw will dump out all results
    "num_spin_reversal_transforms": 0,
    "readout_thermalization": 0,  # default 0
    "programming_thermalization": 1000,  # default 1000
    "chain_strength": 1,
    "anneal_offsets": list(anneal_offset),
}

In [29]:
result = embed.sample_qubo(qubo_dict, **dwave_config)
raw = result.to_pandas_dataframe()

In [30]:
raw

,0,1,chain_break_fraction,energy,num_occurrences
0,0,1,0.0,-0.25,706
1,1,0,0.0,-0.25,268
2,0,0,0.0,0.00,25
3,1,1,0.0,0.50,1
